Code based on https://github.com/blei-lab/variational-smc.

In [4]:
using Random, LinearAlgebra, Distributions

In [1]:
mutable struct LGSS_SMC
    T::Integer     # Length of sequence
    dimX::Integer # Dimension of latent variables
    dimY::Integer # Dimension of observations
    N::Integer     # Number of particles
    
    sim_prop # Function for sampling from proposal
end

In [22]:
# TODO: fix seed

function resampling(W; seed=Random.seed!(0))
    # TODO: this isn't the resampling algorithm described in the paper?
    # Taking indices, adding uniform noise, then dividing by N, and re-fitting them into bins
    N = size(W)[1]
    bins = cumsum(W)
    ind = 0:N-1
    u = (ind + rand(N)) / N
    digitize = i -> searchsortedfirst(bins,i)
    return digitize.(u)
end

function vsmc_lower_bound(prop_params, model_params, y, smc_params; seed=Random.seed(0), verbose=false, adapt_resamp=false)
    # Extract constants
    T = size(y)[1]
    dimX = smc_params.dimX
    N = smc_params.N
    
    # Initialize SMC
    X = zeros(N,dimX)
    Xp = zeros(N,dimX)
    logW = zeros(N)
    W = exp.(logW) ./ N
    logZ = 0
    ESS = (1/sum(W.^2)) / N # Effective sample size
    
    for t in 1:T
        # Resampling
        if adapt_resamp
            # TODO
        else
            if t > 0
                ancestors = resampling(W,seed=seed)
                Xp = getindex(X,ancestors)
            else
                Xp = X
            end
        end
        
        # Propagation
        X = smc_params.sim_prop(t, Xp, y, prop_params, model_params, seed=seed)
    end
    return X
end

vsmc_lower_bound (generic function with 1 method)

In [93]:
K = [1,3,2,4]
getindex(K,[1,1,2])

3-element Array{Int64,1}:
 1
 1
 3

In [23]:
function init_model_params(dimX, dimY, α, r, obs; seed=Random.seed(0))
    μ0 = zeros(dimX)
    Σ0 = Matrix{Float64}(I,dimX,dimX)
    
    A = zeros(dimX,dimY)
    for i in 1:dimX
        for j in 1:dimX
            A[i,j] = α^(abs(i-j)+1)
        end
    end
    
    Q = Matrix{Float64}(I,dimX,dimX)
    
    # TODO: obs
    # C = zeros(dimY,dimX)
    C = randn(dimY,dimY)
    
    R = r*Matrix{Float64}(I,dimY,dimY)
    
    return μ0, Σ0, A, Q, C, R
end

function init_prop_params(T, dimX, scale; seed=Random.seed(0))
    return [(scale*randn(dimX), # Bias
            1 .+ scale*randn(dimX), # Linear times A/μ0
            scale*randn(dimX)) # Log-var
            for t in 1:T]
end

function generate_data(model_params, T; seed=Random.seed(0))
    μ0, Σ0, A, Q, C, R = model_params
    dimX = size(μ0)[1]
    dimY = size(R)[1]
    
    x_true = zeros(T,dimX)
    y_true = zeros(T,dimY)
    
    for t in 1:T
        if t > 1
            x_dist = MvNormal(A*x_true[t-1,:],Q)
        else
            x_dist = MvNormal(μ0,Σ0)
        end
        x_true[t,:] = rand(x_dist,1)
        
        y_dist = MvNormal(C*x_true[t,:],R)
        y_true[t,:] = rand(y_dist,1)
    end
    
    return x_true, y_true
end

function log_marginal_likelihood(model_params, T, y_true)
    μ0, Σ0, A, Q, C, R = model_params
    dimX = size(μ0)[1]
    dimY = size(R)[1]
    
    # Compute via Kalman filter
    log_likelihood = 0
    
    x_filt = zeros(dimX)
    P_filt = zeros(dimX,dimX)
    x_pred = μ0
    P_pred = Σ0
    
    for t in 1:T
        if t > 0
            # Predict
            x_pred = A * x_filt
            P_pred = A*P_filt*A' + Q
        end
        
        # Update
        yt = y_true[t,:] - C*x_pred
        S = C*P_pred*C' + R
        K = (S \ (C*P_pred))'
        x_filt = x_pred + K*yt
        P_filt = P_pred - K*C*P_pred
        
        # TODO: double check this correct
        log_likelihood += -0.5*(dot(yt,S\yt) + logdet(S) + dimY*log(2*π))
    end
    return log_likelihood
end

function sim_prop(t, Xp, y, prop_params, model_params; seed=Random.seed!(0))
    μ0, Σ0, A, Q, C, R = model_params
    μt, lint, log_σ2t = prop_params[t]
    σt = sqrt.(exp.(log_σ2t))
    
    if t > 0
        println(size(A))
        println(size(Xp))
        println(size(lint))
        μ = μt + (A*Xp')'*lint
        println("here2")
    else
        μ = μt + lint*μ0
    end
    return μ + randn(size(Xp))*σt
end

sim_prop (generic function with 1 method)

In [5]:
randn(size(Matrix(I,5,5)))

5×5 Array{Float64,2}:
  0.479685  -0.247706   1.41547    0.304127  -2.09069
  0.297071  -0.822784   1.2586     1.85592    1.0211
  0.235948   1.78808    0.696116  -1.47113   -0.718043
  0.629158   0.869882  -0.593085  -0.983135   1.04007
 -1.25279   -0.146835  -1.87564    1.63912   -0.389665

In [24]:
# Model hyperparameters
T = 2
dimX = 2
dimY = 2
α = 0.42
r = 0.1
obs = "sparse"

# Training parameters
param_scale = 0.5
num_epochs = 1000
step_size = 0.001

N = 4

data_seed = Random.seed!(1)
model_params = init_model_params(dimX, dimY, α, r, obs, seed=data_seed)

x_true, y_true = generate_data(model_params, T, seed=data_seed)

log_marginal = log_marginal_likelihood(model_params, T, y_true)
println("True log-marginal likelihood: ", log_marginal)

seed = Random.seed!(1)

prop_params = init_prop_params(T, dimX, param_scale, seed=seed)
combined_init_params = (model_params, prop_params)

lgss_smc_obj = LGSS_SMC(T,dimX,dimY,N,sim_prop)

vsmc_lower_bound(prop_params, model_params, y_true, lgss_smc_obj, seed=seed)

True log-marginal likelihood: -5.524367060004888
(2, 2)
(4,)
(2,)


LoadError: [91mDimensionMismatch("A has dimensions (2,2) but B has dimensions (1,4)")[39m